In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
import scipy.stats.mstats as gmean
import matplotlib.pyplot as plt
import csv

if __name__ == "__main__":
    dataset = pd.read_csv('ura_data_processed.csv', index_col=0)
    y = dataset['log_price_psf']

In [6]:
import sys
class classImputations():
    def __init__ (self, df):
        try:
            super.__init__()
            self.df = df
        except Exception as e:
            sys.exit(1)

In [2]:
X_columns = [c for c in dataset.columns if not c.startswith('Period_') and not c in ['log_price_psf']]
X = dataset[X_columns]
X = sm.add_constant(X)
period_list = ['%sQ%s' % (year, qtr) for year in range(2016,2022) for qtr in range(1,5)]
period_list = period_list[:-3]
print(period_list)

['2016Q1', '2016Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q2', '2017Q3', '2017Q4', '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1']


In [90]:
import math
debug_predictions = {}
for period in period_list:
    with open('%s.pkl' % period, 'rb') as file:
        results = pickle.load(file)

    prediction = results.predict(X)
    prediction = prediction.astype(float)
    dataset['pred_psf_%s' % period] = np.exp(prediction)

pred_columns = [c for c in dataset.columns if c.startswith('pred_')]
base_period = '2016Q1'
base_period_filter = dataset['Period_%s' % base_period]
imputation_index_laspeyres = dataset.loc[base_period_filter,
                                         pred_columns].mean() / dataset[base_period_filter][pred_columns].mean()[0]

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_555/3433398503.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_columns].mean() / dataset[base_period_filter][pred_columns].mean()[0]


In [91]:
imputation_index_laspeyres.index = [x.replace('pred_psf_', '') for x in imputation_index_laspeyres.index]

In [92]:
district_9_filter = dataset['District_9'] == 1
imputation_index_9_laspeyres = dataset.loc[base_period_filter & 
                                           district_9_filter, pred_columns].mean() / dataset.loc[base_period_filter & 
                                                                                                 district_9_filter, pred_columns].mean()[0]
imputation_index_9_laspeyres.index = [x.replace('pred_psf_', '') for x in imputation_index_9_laspeyres.index]

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_555/3371077177.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  district_9_filter, pred_columns].mean() / dataset.loc[base_period_filter &


In [93]:
paasche_index_dict = {}
paasche_index_9_dict = {}
for period in period_list:
    period_filter = dataset['Period_%s' % period] == 1
    paasche_index_dict[period] = dataset.loc[period_filter]['pred_psf_%s' % period].mean() / dataset[period_filter]['pred_psf_%s' % base_period].mean()
    paasche_index_9_dict[period] = dataset.loc[period_filter & district_9_filter]['pred_psf_%s' % period].mean() / dataset[period_filter & district_9_filter]['pred_psf_%s' % base_period].mean()

In [96]:
##Imputation Index
imputation_index_paasche = pd.DataFrame.from_dict(paasche_index_dict, orient='index', columns = ['paasche_all'])